In [15]:
from IPython import display

from matplotlib import pyplot as plt
%matplotlib inline

import math, itertools

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

## Load Data

In [2]:
# No rescaling needed. MNIST data is in range [0, 1]
mnist = input_data.read_data_sets("tf_data/")

Extracting tf_data/train-images-idx3-ubyte.gz
Extracting tf_data/train-labels-idx1-ubyte.gz
Extracting tf_data/t10k-images-idx3-ubyte.gz
Extracting tf_data/t10k-labels-idx1-ubyte.gz


## Model

In [3]:
# Discriminator Neural Network
class Discriminator(object):
    def __init__(self):
        """ Init the model with hyper-parameters etc. """
        # Layer 1 Variables
        self.weights1 = tf.Variable(tf.zeros([784, 1024]), name='weights1')
        self.biases1 = tf.Variable(tf.zeros([1024]), name='biases1')
        # Layer 2 Variables
        self.weights2 = tf.Variable(tf.zeros([1024, 512]), name='weights2')
        self.biases2 = tf.Variable(tf.zeros([512]), name='biases2')
        # Layer 3 Variables
        self.weights3 = tf.Variable(tf.zeros([512, 256]), name='weights3')
        self.biases3 = tf.Variable(tf.zeros([256]), name='biases3')
        # Out Layer Variables
        self.weights4 = tf.Variable(tf.zeros([256, 1]), name='weights4')
        self.biases4 = tf.Variable(tf.zeros([1]), name='biases4')
        # Store Variables in list
        self.var_list = [self.weights1, self.biases1, self.weights2, self.biases2, 
                           self.weights3, self.biases3, self.weights4, self.biases4]
            
    def inference(self, x):
        """ This is the forward calculation from x to y """
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights1) + self.biases1), .3)
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights2) + self.biases2), .3)
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights3) + self.biases3), .3)
        x = tf.nn.sigmoid(tf.matmul(x, self.weights4) + self.biases4)
        return x
    
    def loss(self, inference_real, inference_fake):
        return -tf.reduce_mean(tf.log(inference_real) + tf.log(1. - inference_fake), name='D_loss')
        
    def optimize(self, loss):
        return tf.train.AdamOptimizer().minimize(loss, name='D_optimizer', var_list=self.var_list)

In [4]:
# Geneator Neural Network
class Generator(object):
    def __init__(self):
        self._init_variables()
            
    def _init_variables(self):
        with tf.variable_scope("G"):
            """ Init the model with hyper-parameters etc. """
            # Layer 1 Variables
            self.weights1 = tf.Variable(tf.zeros([100, 256]))
            self.biases1 = tf.Variable(tf.zeros([256]))
            # Layer 2 Variables
            self.weights2 = tf.Variable(tf.zeros([256, 512]))
            self.biases2 = tf.Variable(tf.zeros([512]))
            # Layer 3 Variables
            self.weights3 = tf.Variable(tf.zeros([512, 1024]))
            self.biases3 = tf.Variable(tf.zeros([1024]))
            # Out Layer Variables
            self.weights4 = tf.Variable(tf.zeros([1024, 784]))
            self.biases4 = tf.Variable(tf.zeros([784]))
            # Store Variables in list
            self.var_list = [self.weights1, self.biases1, self.weights2, self.biases2, 
                               self.weights3, self.biases3, self.weights4, self.biases4]
            
    def inference(self, x):
        """ This is the forward calculation from x to y """
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights1) + self.biases1)
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights2) + self.biases2)
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights3) + self.biases3)
        x = tf.nn.tanh(tf.matmul(x, self.weights4) + self.biases4)
        return x
    
    def loss(self, inference, discriminator):
        G_fake_images_prob = discriminator.inference(inference)
        return -tf.reduce_mean(tf.log(G_fake_images_prob), name='G_loss')
        
    def optimize(self, loss):
        # Add a scalar summary for the snapshot loss.
        tf.summary.scalar('G_loss', loss)
        # Create Adam optimizer
        optimizer = tf.train.AdamOptimizer()
        # Use optimizer to apply gradients
        train_op = optimizer.minimize(loss, name='G_optimizer', var_list=self.var_list)
        return train_op

In [5]:
def noise(n_rows, n_cols):
    return np.random.normal(size=(n_rows, n_cols ))

## Initialize Graph

In [6]:
IMAGE_PIXELS = 28*28
NOISE_SIZE = 100

In [7]:
# Init Placeholders
D_images_real = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
D_images_fake = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
# Initialize D
discriminator = Discriminator()
# Build a Graph that computes D
D_inference_real = discriminator.inference(D_images_real)
D_inference_fake = discriminator.inference(D_images_fake)
# Add to D's graph the Ops for loss calculation.
D_loss = discriminator.loss(D_inference_real, D_inference_fake)
# Add the Op to optimize D's variables
D_optim = discriminator.optimize(D_loss)

In [8]:
# Init Placeholders
G_noise = tf.placeholder(tf.float32, shape=(None, NOISE_SIZE))
# Initialize G.
generator = Generator()
# Build a Graph that computes G
G_inference = generator.inference(G_noise)
# Add to the Graph the Ops for loss calculation.
G_loss = generator.loss(G_inference, discriminator)
# Add the Op to optimize G's variables
G_optim = generator.optimize(G_loss)

## Train

In [9]:
BATCH_SIZE = 100
NUM_EPOCHS = 100000

In [ ]:
# create figure for plotting
size_figure_grid = int(math.sqrt(16))
fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)


# Start interactive session
session = tf.InteractiveSession()
# Init Variables
tf.global_variables_initializer().run()

# Iterate through epochs
for i in range(NUM_EPOCHS):

    # Train Discriminator
    real_images = mnist.train.next_batch(batch_size=BATCH_SIZE)[0]
    fake_images = session.run(G_inference, feed_dict={G_noise: noise(BATCH_SIZE,NOISE_SIZE)})
    feed_dict = {D_images_real: real_images, D_images_fake: fake_images}
    _, D_loss_i = session.run([D_optim, D_loss], feed_dict=feed_dict)
    
    # Train Generator
    feed_dict = {G_noise: noise(BATCH_SIZE,NOISE_SIZE)}
    _, G_loss_i, G_inference_i = session.run([G_optim, G_loss, G_inference], feed_dict=feed_dict)
    
    if (i) % 100 == 0:
        display.clear_output(True)
        for k in range(16):
            i = k//4
            j = k%4
            ax[i,j].cla()
            ax[i,j].imshow(G_inference_i[k,:].reshape(28, 28), cmap='Greys')
        display.display(plt.gcf())
        print(D_loss_i, G_loss_i)